In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


X, y = fetch_openml('Fashion-MNIST', version=1, return_X_y=True)
X /= 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linearization
x_train = np.array(X_train).reshape(X_train.shape[0], -1)
x_test = np.array(X_test).reshape(X_test.shape[0], -1)

x_train.shape, x_train.dtype, x_test.shape, x_test.dtype, X.shape

((56000, 784), dtype('float64'), (14000, 784), dtype('float64'), (70000, 784))

In [2]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [3]:
class_names = [
    'T-shirt',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

class_names[int(y_train[0])]

'Ankle boot'

In [4]:
from sklearn.cluster import KMeans


clusterizer = KMeans(n_clusters=10, random_state=0).fit(x_train)
clusterizer.predict(x_test)

array([0, 2, 1, ..., 7, 3, 2], dtype=int32)

In [5]:
from sklearn.cluster import BisectingKMeans


clusterizer = BisectingKMeans(n_clusters=10, init='k-means++', random_state=0, 
                              bisecting_strategy='largest_cluster').fit(x_train)
clusterizer.predict(x_test)

array([4, 9, 0, ..., 5, 1, 9], dtype=int32)

In [ ]:
from sklearn.cluster import OPTICS


clusterizer = OPTICS(min_samples=4600, n_jobs=-1).fit(x_train)
clusterizer.labels_